In [9]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

## Загружаем датасет

In [4]:
df = pd.read_csv('../releases/release2/data.csv')
df.head()

,id,num_rooms,metro,address,area_m2,parking,price,repair,balcony
0,271271157,4.0,Смоленская,Москва,200.0/20.0,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,нет
1,271634126,4.0,Смоленская,Москва,198.0/95.0/18.0,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Дизайнерский,нет
2,271173086,4.0,Смоленская,Москва,200.0/116.0/4.0,подземная,"500000.0 руб./ За месяц, Залог - 500000 руб., ...",Евроремонт,нет
3,272197456,4.0,Смоленская,Москва,170.0/95.0/17.0,подземная,"400000.0 руб./ За месяц, Залог - 400000 руб., ...",Евроремонт,нет
4,273614615,2.0,Арбатская,Москва,58.0/38.0/5.0,нет,"225000.0 руб./ За месяц, Залог - 225000 руб., ...",Евроремонт,нет


## В "area_m2" оставим только одно число

In [5]:
df['area_m2'] = df['area_m2'].apply(lambda x: float(x.split("/")[0]) if type(x) is str else x)
df['area_m2']

0        200.0
1        198.0
2        200.0
3        170.0
4         58.0
         ...  
22048     35.0
22049     38.7
22050     43.1
22051     52.5
22052     90.0
Name: area_m2, Length: 22053, dtype: float64

## В "price" оставим только цену аренды, а предыдущее значение скопируем в другой столбец для последующих операций

In [6]:
df['rent'] = df['price']
df['price'] = pd.to_numeric(df['price'].apply(lambda x: x.split()[0]))
df['price']

0        500000.0
1        500000.0
2        500000.0
3        400000.0
4        225000.0
           ...   
22048     42000.0
22049     45000.0
22050     50000.0
22051     55000.0
22052     57000.0
Name: price, Length: 22053, dtype: float64

## Из нового столбца возьмем информацию о коммунальных услугах

In [7]:
df['rent'] = df['rent'].apply(lambda x: "yes" if "Коммунальные услуги включены" in x else "no")
df['rent']

0        yes
1        yes
2         no
3         no
4        yes
        ... 
22048    yes
22049    yes
22050    yes
22051    yes
22052    yes
Name: rent, Length: 22053, dtype: object

## На этом мы решили закончить Feature engineering, но есть некоторые идеи по его дополнению:
## Добавить колонку с расстоянием до центра города, конверитруя Адрес в координаты и вычисляя расстояние;
## И используя те же координаты создать колонку района города, в котором находится дом;
## Также в удаленном столбце Описание есть много неоднородной информации, например слова: люкс, особняк и т.д., этот столбец можно подать на вход языковой модели для выявления данных слов и потом уже работать с выжимкой из Описания.

## Далее закодируем категориальные признаки

In [10]:
for i in ['metro', 'address', 'parking', 'repair', 'balcony', 'rent']:
    label_encoder = LabelEncoder()
    df[i] = label_encoder.fit_transform(df[i])
df.head()

,id,num_rooms,metro,address,area_m2,parking,price,repair,balcony,rent
0,271271157,4.0,239,0,200.0,5,500000.0,1,18,1
1,271634126,4.0,239,0,198.0,5,500000.0,1,18,1
2,271173086,4.0,239,0,200.0,5,500000.0,2,18,0
3,272197456,4.0,239,0,170.0,5,400000.0,2,18,0
4,273614615,2.0,12,0,58.0,3,225000.0,2,18,1


## Уберем строки с выбросами цены

In [13]:
q_low = df["price"].quantile(0.01)
q_hi  = df["price"].quantile(0.99)

df_filtered = df[(df["price"] < q_hi) & (df["price"] > q_low)]

## Теперь нормализуем наши данные

## Разница по цене с выбросами и без значительная

In [11]:
normalized_df=(df-df.min())/(df.max()-df.min())
normalized_df.head()

,id,num_rooms,metro,address,area_m2,parking,price,repair,balcony,rent
0,0.977727,0.6,0.778502,0.0,0.240050,1.0,0.165275,0.333333,1.0,1.0
1,0.979892,0.6,0.778502,0.0,0.237562,1.0,0.165275,0.333333,1.0,1.0
2,0.977143,0.6,0.778502,0.0,0.240050,1.0,0.165275,0.666667,1.0,0.0
3,0.983251,0.6,0.778502,0.0,0.202736,1.0,0.131886,0.666667,1.0,0.0
4,0.991701,0.2,0.039088,0.0,0.063433,0.6,0.073456,0.666667,1.0,1.0


In [14]:
normalized_df=(df_filtered-df_filtered.min())/(df_filtered.max()-df_filtered.min())
normalized_df.head()

,id,num_rooms,metro,address,area_m2,parking,price,repair,balcony,rent
0,0.977727,0.6,0.778502,0.0,0.243380,1.0,0.842547,0.333333,1.0,1.0
1,0.979892,0.6,0.778502,0.0,0.240858,1.0,0.842547,0.333333,1.0,1.0
2,0.977143,0.6,0.778502,0.0,0.243380,1.0,0.842547,0.666667,1.0,0.0
3,0.983251,0.6,0.778502,0.0,0.205549,1.0,0.667600,0.666667,1.0,0.0
4,0.991701,0.2,0.039088,0.0,0.064313,0.6,0.361442,0.666667,1.0,1.0


## Сохраним полученный датасет

In [15]:
normalized_df.to_csv('data1.csv', index=False)